In [2]:
# week1 -> day1
import os
from dotenv import load_dotenv
from openai import OpenAI

#week2 -> day2
import gradio as gr

In [3]:
load_dotenv(override=True)
api_key:str = os.getenv('OPENAI_API_KEY')

In [26]:
class SolveTechnicalQuestions:
    _system_prompt = """
        You are a snarkyassistant that analyzes the contents of a website, 
        and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
        Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
    """
    _stream = False

    def __init__(self, model: str = "gpt-4o-mini") -> None:
        self.openai_client = OpenAI()
        self._chat_llm = OpenAI()
        self._MODEL = model
    
    def set_system_prompt(self, system_prompt: str) -> None:
        self._system_prompt = system_prompt 

    def set_stream(self, stream: bool) -> None:
        self._stream = stream
        
    def set_endpoint(self, endpoint: str, api_key: str = "ollama") -> None:
        self.openai_client = OpenAI(base_url=endpoint, api_key=api_key)

    def set_model(self, model: str) -> None:
        if model == "GPT" and self._MODEL != "gpt-4o-mini":
            self._chat_llm = OpenAI()
            self._MODEL = "gpt-4o-mini"
        elif model == "ollama" and self._MODEL != "llama3.2":
            self._chat_llm = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
            self._MODEL = "llama3.2"

    def talker(self, message):
        response = self.openai_client.audio.speech.create(
        model="gpt-4o-mini-tts",
        voice="onyx",    # Also, try replacing onyx with alloy or coral
        input=message
        )
        return response.content

    def initiate_chat(self, message, history):
        return "", history + [{"role":"user", "content":message}]

    def audio_to_text(self, audio_file, history):
        """Convert audio file to text using OpenAI Whisper"""

        
        result = history + [{"role": "user", "content": ""}]
        try:
            if audio_file is None:
                print("No audio file provided")
                result[-1]["content"] = "No speech detected in audio"
                return result
            
            # Ensure we have the file path
            if isinstance(audio_file, str):
                file_path = audio_file
            else:
                file_path = audio_file.name if hasattr(audio_file, 'name') else str(audio_file)
            
            # Check if file exists
            if not os.path.exists(file_path):
                print(f"Audio file not found: {file_path}")
                result[-1]["content"] = "No speech detected in audio"
                return result
            
            # Check file size (Whisper has limits)
            file_size = os.path.getsize(file_path)
            if file_size > 25 * 1024 * 1024:  # 25MB limit
                result[-1]["content"] = "Audio file too large (max 25MB)"
                return result
            
            # Transcribe using OpenAI Whisper
            with open(file_path, "rb") as audio:
                response = self.openai_client.audio.transcriptions.create(
                    model="whisper-1",
                    file=audio,
                    response_format="text"
                )
            
            # Clean up the transcribed text
            text = response.strip()
            
            if not text:
                result[-1]["content"] ="No speech detected in audio"
                return result
            
            result[-1]["content"] = text
            return result
            
        except Exception as e:
            error_msg = f"Audio transcription error: {str(e)}"
            print(f"{error_msg}")
            result[-1]["content"] = "No speech detected in audio"
            return result

    def chat(self, history):
        history = [{"role": h["role"], "content": h["content"]} for h in history]
        messages = [{"role": "system", "content": self._system_prompt}] + history
        stream = self._chat_llm.chat.completions.create(model=self._MODEL, messages=messages, stream=self._stream)

        if self._stream:
            response = ""
            voice = None

            for chunk in stream:
                if chunk.choices[0].delta.content:
                    response += chunk.choices[0].delta.content
                    temp_history = history + [{"role": "assistant", "content": response}]
                    yield temp_history, voice 
            
            voice = self.talker(response)
            history += [{"role": "assistant", "content": response}]
            yield history, voice
        else:
            response = stream.choices[0].message.content
            history += [{"role": "assistant", "content": response}]
            voice = self.talker(response)
            yield history, voice

    def Interface(self, title, name, desc):
        with gr.Blocks(title=title) as ui:
            with gr.Column():
                gr.Markdown(f"""
                <div style="text-align: center; padding: 10px 10px;">
                    <h1 style="color: white; margin: 0; font-size: 42px; font-weight: bold;">✈️ {name}</h1>
                    <p style="color: #7f8c8d; font-size: 12px; margin: 20px 0 0 0; max-width: 700px; margin-left: auto; margin-right: auto; line-height: 1.6;">
                        {desc}
                    </p>
                </div>
                """)
            with gr.Row():
                chatbot = gr.Chatbot(height=500, type="messages")
            with gr.Row():
                audio_output = gr.Audio(autoplay=True)
            with gr.Row():
                mic = gr.Mic(label="Talk to the AI Assistant", type="filepath", editable=False)
                with gr.Column():
                    submit = gr.Button("Submit", variant="primary")
                    stream = gr.Checkbox(label="Stream", value=False)
                model = gr.Dropdown(choices=["GPT", "ollama"], value="GPT", label="Model")
                message = gr.Textbox(label="Chat with our AI Assistant:")

            model.change(fn=self.set_model, inputs=model)
            stream.change(fn=self.set_stream, inputs=stream)
            submit.click(fn=self.audio_to_text, inputs=[mic, chatbot], outputs=chatbot).then(
                fn=self.chat, inputs=chatbot, outputs=[chatbot, audio_output]
            )
            message.submit(fn=self.initiate_chat, inputs=[message, chatbot], outputs=[message, chatbot]).then(
                fn=self.chat, inputs=chatbot, outputs=[chatbot, audio_output]
            )

        ui.launch()






In [31]:
TECHNICAL_SYSTEM_PROMPT = """
You are a knowledgeable and friendly technical assistant specializing in programming, systems, and AI/ML. You're approachable and conversational while staying focused on your technical expertise.

EXPERTISE AREAS:
- Programming & Development (Python, JavaScript, Java, C++, Go, Rust, TypeScript)
- Web Development (React, Vue, Angular, Node.js)
- Mobile Development (iOS, Android, Flutter)
- DevOps (Docker, Kubernetes, CI/CD, AWS, Azure, GCP)
- Database Systems (SQL, NoSQL, PostgreSQL, MongoDB)
- Software Architecture & Best Practices
- Systems & Infrastructure (Linux, Windows, macOS, Networking, Security)
- Cloud Computing & Distributed Systems
- AI & Machine Learning (TensorFlow, PyTorch, NLP, Computer Vision, MLOps)

CONVERSATION STYLE:
- Be friendly and respond to greetings warmly
- Acknowledge casual questions briefly but redirect to technical topics
- Ask clarifying questions about technical problems
- Provide accurate, up-to-date technical information
- Include code examples when relevant
- Explain complex concepts clearly
- Suggest best practices and alternatives

RESPONSE GUIDELINES:
1. Respond to greetings and casual conversation politely
2. For non-technical questions, briefly acknowledge them and offer to help with technical topics
3. Focus primarily on technical questions within your expertise
4. Provide practical, actionable technical advice
5. Reference official documentation when appropriate
6. Be encouraging and supportive for technical learning

REDIRECTION EXAMPLES:
- "That's an interesting question! While I'm here to help with technical topics, I'd love to assist you with any programming, systems, or AI questions you might have. What technical challenge can I help you with today?"
- "I'm specialized in technical assistance. How can I help you with programming, development, or systems questions?"

Remember: Be friendly but stay focused on your technical expertise. Redirect non-technical questions back to your core competencies.
"""

Chat = SolveTechnicalQuestions()
Chat.set_system_prompt(TECHNICAL_SYSTEM_PROMPT)

In [ ]:
Chat.Interface("Technical Assistant", "Techie", "Ask me anything about technology!")